In [1]:
import logging
from typing import Any, Set

from automata.config.base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.context_providers.symbol_synchronization import (
    SymbolProviderSynchronizationContext,
)
    
logger = logging.getLogger(__name__)

py_module_loader.initialize()

In [2]:
symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [3]:
raw_ranks = symbol_search.symbol_rank.get_ranks()
print("Demonstrating SymbolRank\n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 661/661 [00:02<00:00, 265.99it/s]

Demonstrating SymbolRank

rank 0 = automata.symbol.base.Symbol with rank 0.006
rank 1 = automata.core.base.database.vector.ChromaVectorDatabase with rank 0.005
rank 2 = automata.agent.providers.OpenAIAutomataAgent with rank 0.005
rank 3 = automata.core.base.database.vector.VectorDatabaseProvider with rank 0.005
rank 4 = automata.symbol.graph.SymbolGraph with rank 0.004
rank 5 = automata.core.base.database.vector.JSONVectorDatabase with rank 0.004
rank 6 = automata.retrievers.py.context.PyContextRetriever with rank 0.004
rank 7 = automata.singletons.dependency_factory.DependencyFactory with rank 0.004
rank 8 = automata.symbol_embedding.vector_databases.ChromaSymbolEmbeddingVectorDatabase with rank 0.004
rank 9 = automata.singletons.github_client.RepositoryClient with rank 0.004


In [4]:
query = "SymbolGraph"
symbol_rank_search_results = symbol_search.symbol_rank_search(query)
print("Demonstrating SymbolRank results for query = `SymbolGraph`\n")
raw_ranks = symbol_search.symbol_rank.get_ranks()
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

Demonstrating SymbolRank results for query = `SymbolGraph`

rank 0 = automata.symbol.graph.SymbolGraph with rank 0.015
rank 1 = automata.symbol.base.Symbol with rank 0.011
rank 2 = automata.symbol.graph.GraphBuilder with rank 0.009
rank 3 = automata.retrievers.py.context.PyContextRetriever with rank 0.007
rank 4 = automata.singletons.dependency_factory.DependencyFactory with rank 0.007
rank 5 = automata.experimental.search.symbol_search.SymbolSearch with rank 0.007
rank 6 = automata.experimental.search.rank.SymbolRank with rank 0.007
rank 7 = automata.symbol.graph.GraphBuilder.build_graph with rank 0.006
rank 8 = automata.symbol.graph._CallerCalleeProcessor with rank 0.006
rank 9 = automata.symbol.graph._RelationshipProcessor with rank 0.006


In [5]:
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_ordered_entries()

print("Demonstrating code embedding search results for query = `SymbolGraph`\n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with distance {rank[1]:.3f}")

Demonstrating code embedding search results for query = `SymbolGraph`

rank 0 = automata.symbol.graph.SymbolGraph with distance 0.797
rank 1 = automata.singletons.dependency_factory.DependencyFactory.create_symbol_graph with distance 0.769
rank 2 = automata.symbol.graph.GraphBuilder with distance 0.768
rank 3 = automata.symbol.graph.GraphBuilder.build_graph with distance 0.758
rank 4 = automata.singletons.dependency_factory.DependencyFactory.create_subgraph with distance 0.755
rank 5 = automata.symbol.graph._RelationshipProcessor.process with distance 0.749
rank 6 = automata.symbol.graph.SymbolGraph.__init__ with distance 0.747
rank 7 = automata.symbol.graph._CallerCalleeProcessor.__init__ with distance 0.747
rank 8 = automata.symbol.graph._CallerCalleeProcessor.process with distance 0.746
rank 9 = automata.symbol.graph._RelationshipProcessor with distance 0.745
